#### Housekeeping (run once per kernel restart)

In [ ]:
# change directory to parent
import os
os.chdir('..')
print(os.getcwd())

# Imports

In [ ]:
import importlib
import itertools
import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import torch

import minimal_environment as me
importlib.reload(me)

# Environments

Environments are defined as discrete time Partially Observable Markov Decision Processes ([POMDP](https://en.wikipedia.org/wiki/Partially_observable_Markov_decision_process)s) witout reward function:

- a set of states $s \in \mathcal{S}$
- a set of observations $o \in \mathcal{\Omega}$
- a set of actions $a \in \mathcal{A}$
- a set of conditional transition probabilities $\mathcal{\tau}: p(s'|s, a)$
- a set of emission/ observation probabilities $\mathcal{O}: p(o|s)$

## MinimalEnv environment

The `MinimalEnv` environment has a discrete set of states and generates discrete (binary) outputs: food `True` or no food `False`.

### Emission probability 
The emission probability distributions $p(o|s)$ are defined as a conditional probability table `p[s,o]`, where each row $i$ defines the probability of observing `False` (column 0) and `True` (column 1) in state $s_i$.

In [ ]:
env = me.MinimalEnv(N=16, # number of states
                    s_food=0, # location of the food source
                    o_decay=0.4) # decay of observing food away from source 

p_o_given_s = env.p_o_given_s #precomputed result of env.emission_probability()
print(p_o_given_s)
print('shape', p_o_given_s.shape)

fig, ax = plt.subplots(figsize=(8, 3))
o_food = 1
ax.bar(range(env.p_o_given_s[:,1].shape[0]), env.p_o_given_s[:,o_food])
ax.bar([env.s_food], env.p_o_given_s[env.s_food,o_food], color='red', label='food source')
ax.set_xlabel('state')
ax.set_ylabel('$p(o=True|s)$')
ax.legend()

### Transition dynamics
The transition dynamics $p(s'|s, a)$ are defined as a conditional probability table `p[s,a,s']`. The subarray `p[0,0,:]`, for example, defines the probability of transitioning from state $0$ to any successor state, given that action `0` (move left) was taken.

In [ ]:
p_s1_given_s_a = env.p_s1_given_s_a # precomputed result of env.transition_dynamics()
print('shape', p_s1_given_s_a.shape)
p_s1_given_s_a[0,0,:] # left: 0, right: 1

### Random Agent Behavior

Unlike a POMDP, the environment itself does not define a goal, motivation, or purpose for an agent that interacts with it. The envirment is indifferent about how agents interact with it. There is therefore no value (good, bad, high or low performance) associated with any individual sequence of behavior. 

The code below simulates the interaction between the minimal environment and an agent that behaves randomly.

In [ ]:
n_steps = 100
ss, os = [], []

o = env.reset()
ss.append(env.s_t)
os.append(o)

for i in range(n_steps):
  a = np.random.choice([0,1]) # random agent
  o = env.step(a)
  ss.append(env.s_t)
  os.append(o)

We inspect the sequence of states and emissions during this interaction.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16, 12))
ax[0].plot(ss, label='agent state $s_t$')
ax[0].plot(np.ones_like(ss) * env.s_food, 
           'r--', label='food source', linewidth=1)
ax[0].set_xlabel('timestep t')
ax[0].set_ylabel('$s$')
ax[0].legend()
ax[1].plot(np.array(os))
ax[1].set_xlabel('timestep t')
ax[1].set_ylabel('observation (1=Food)')

# Active Inference

## Belief Update

An active inference agent holds a belief about the state of the environment at time $t$, modelled as a probability distribution $Q(s; \theta_t)$ over states $s \in \mathcal{S}$. There is uncertainty associated with this belief as the state cannot be observed directly and must instead be inferred from stochastic observations.

### Update through Time

At any time $t$ an agent holds a prior belief about $s_t$ before taking in an observation from its environment. This belief could be uniform, for example at the start of an interaction. It could also be informed by propagating the belief $Q(s; \theta_{t-1})$ about state $s_{t-1}$ through its model of the environment transition dynamics, taking into account the action $a_{t-1}$ taken at the previous time step.

$$Q(s; \theta_{t}) = \mathbb{E}_{s\sim Q_{t-1}}[p(s_t|s, a_{t-1})]$$

Note that updating the belief in light of the chosen action involves fitting the parameters $\theta_t$. For our minimal agent, we represent $Q(s;\theta)$ as a softmax with logits $\theta$. For diagnostics and debugging, we optionally log the optimization loss (`debug=True`)



In [ ]:
def update_belief_a(env, theta_prev, a, lr=4., n_steps=10, debug=False):
    # prior assumed to be expressed as parameters of the softmax (logits)
    theta = torch.tensor(theta_prev)
    q = torch.nn.Softmax(dim=0)(theta)
    
    # this is the prior for the distribution at time t
    # if we worked on this level, we would be done. 
    # but we need to determine the parameters of Q that produce 
    # this distribution
    q1 = torch.matmul(q, torch.tensor(env.p_s1_given_s_a[:,a,:]))

    # initialize updated belief to uniform
    theta1 = torch.zeros_like(theta, requires_grad=True)
    loss = torch.nn.CrossEntropyLoss() # expects logits and target distribution.
    optimizer = torch.optim.SGD([theta1], lr=lr)
    if debug:
        ll = np.zeros(n_steps)
        
    for i in range(n_steps):
        l = loss(theta1, q1)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        if debug:
            ll[i] = l.detach().numpy()
            
    theta1 = theta1.detach().numpy()
    if debug:
        return theta1, ll
        
    return theta1

Let's see the effect of this belief update in action. Say we believed strongly that the environment was in states 1 or 4 with equal probability at time $t$ and we took action 1 (move right). Recall that according to the environment dynamics there is some probability of the state remaining unchanged.



In [ ]:
env = me.MinimalEnv(N=8,s_food=0)

theta = np.eye(env.s_N)[1] * 2 + np.eye(env.s_N)[4] * 2
theta1, ll = update_belief_a(env, theta, a=1, 
                             lr=4.0, n_steps=20, debug=True)

def softmax(x):
  e = np.exp(x - x.max())
  return e / e.sum()

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
plt.sca(ax[0])
plt.plot(ll)
plt.plot([0, ll.shape[0]-1], [ll.min()]*2, 'k--')
plt.xlabel('optimization step')
plt.ylabel('loss')

plt.sca(ax[1])
plt.bar(np.arange(env.s_N)-0.2, width=0.4, height=softmax(theta), alpha=0.5, label='before') # belief before update
plt.bar(np.arange(env.s_N)+0.2, width=0.4, height=softmax(theta1), alpha=0.5, label='after') # belief before update
plt.xlabel('env state')
plt.ylabel('belief')
plt.title('Propagating prior beliefs through environment dynamics.')
plt.legend()

### Update based on new observation

Now that we have updated our prior taking into account the action taken in the previous time step and the (agent's model of the) environment dynamics, we turn our attention to updating beliefs in light of a new observation.

In active inference, this belief update is cast as minimizing the variational free energy, i.e. minimizing the KL-divergence between $Q(s;\theta')$ and $p(o, s) = Q(s; \theta) p(o|s)$ with respect to $\theta'$.

$$D_{KL}(\quad Q_{\theta'}(s), p(o|s)Q_{\theta}(s) \quad) \quad = \quad \mathbb{E}_{s \sim Q_{\theta'}}[\quad \log Q_{\theta'}(s) - \log p(o|s)Q_{\theta}(s) \quad]$$

*See book p. 28, Eq. (2.5).*

Again, for diagnostics and debugging, we optionally log the optimization loss (`debug=True`)

In [ ]:
def update_belief(env, theta_prev, o, lr=4., n_steps=10, debug=False):
    theta = torch.tensor(theta_prev)
    
    # make p(s) from b
    q = torch.nn.Softmax(dim=0)
    p = torch.tensor(env.p_o_given_s[:,o]) * q(theta) # p(o|s)p(s)
    log_p = torch.log(p)
    
    # initialize updated belief with current belief (this is an arbitrary choice)
    theta1 = torch.tensor(theta_prev, requires_grad=True)
    
    # estimate loss
    def forward():
        q1 = q(theta1)
        # free energy: KL[ q(s) || p(s, o) ]
        fe = torch.sum(q1 * (torch.log(q1) - log_p))
        return fe
    
    optimizer = torch.optim.SGD([theta1], lr=lr)
    ll = np.zeros(n_steps)
    for i in range(n_steps):
        l = forward()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        if debug:
            ll[i] = l.detach().numpy()
            
    theta1 = theta1.detach().numpy()
    if debug:
        return theta1, ll
        
    return theta1



Let's see the effect of this in action. We start with a uniform prior belief. Recall that the food source is in state 0 and that the probability of observing food decreases exponentially with the distance of a state from the food source, with the state space wrapping around.

If we observed no food ($o=0$), then it is most likely that we are in the state furthest away from the food source. If we observed food ($o=1$), then it is most likely that we are at the food source.

In [ ]:
o = 1 # what do we observe?

env = me.MinimalEnv(N=8,s_food=0)
theta = np.zeros(env.s_N) # uniform prior belief
theta1, ll = update_belief(env, theta, o=o, lr=4., n_steps=20, debug=True)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
plt.sca(ax[0])
plt.plot(ll)
plt.plot([0, ll.shape[0]-1], [ll.min()]*2, 'k--')
plt.xlabel('optimization step')
plt.ylabel('loss')

plt.sca(ax[1])
plt.bar(np.arange(env.s_N)-0.2, width=0.4, height=softmax(theta), alpha=0.5, label='before') # belief before update
plt.bar(np.arange(env.s_N)+0.2, width=0.4, height=softmax(theta1), alpha=0.5, label='after') # belief before update
plt.xlabel('env state')
plt.ylabel('belief')
plt.title('Updating beliefs in light of a new observation.')
plt.legend()

### Accumulating observations over time

Note that, based on a single observation, we cannot refine our belief to become most confident in a state that is not the food source itself or furthest away from the food source. This can be improved if we accumulate observations from a single state over time. Because the probability of observing food decreases symmetrically to the left and right of the food source, the agent is likely to hold strong beliefs about all states that share the same rate of observing food, but cannot disambiguate these states without action.

In [ ]:
s = 3 # true location
n_timesteps = 50

env = me.MinimalEnv(N=12, # number of states
                    s_food=0, # location of the food source
                    s_0=s) # starting location

b = np.zeros(env.s_N) # initialize state prior as uniform
o = env.reset() # set state to starting state

# refine belief by sampling N observations but without taking any action
qq = []
for i in range(n_timesteps):
    b = update_belief(env, theta_prev=b, o=int(o))
    qq.append(softmax(b))
    o = env.sample_o()

plt.bar(range(env.s_N), qq[int(len(qq)*0.25)], alpha=0.3, label='25% observations');
plt.bar(range(env.s_N), qq[-1], alpha=0.5, label='all observations');
plt.legend()

## Action Selection

In active inference, sequences of actions (plans, policies) are scored by the negative expected free energy, and selected by exponentiating and normalizing, i.e. sampling from the softmax over plans. Plans $\pi: a_0, a_1, ..., a_{K-1}$ define sequences of actions up to a finite horizon of $K$ timesteps into the future.

The expected free energy can be decomposed in various ways and here we chose one that we find most intuitive, involving a _pragmatic_ term and an _information gain_ term (See Eq 4.9, page 73 in the book). 

The _pragmatic_ term assesses the probability of arriving in states following $\pi$ that the agent desires, or of encountering observations that the agent desires. In this context, $Q_\theta$ is estimated by propagating beliefs through the environment transition dynamics following the sequence of actions defined by $\pi$, and observations are halucinated by sampling from the emission probability distributions.

$\mathbb{E}_{s \sim Q_{\theta}}\left[\log p_c(s)\right] \quad \text{or} \quad \mathbb{E}_{s \sim Q_{\theta}, o \sim p(o|s)}\left[\log p_c(o)\right]$

The _information gain_ term quantifies the belief update due to making observations in future states.

$\mathbb{E}_{s \sim Q_{\theta}, o \sim p(o|s)}\left[ D_{KL}(\, Q_{\theta'}(s|o),  Q_{\theta}(s) \,) \right]$



In [ ]:
def kl(a, b):
    """ Discrete KL-divergence """
    return (a * (np.log(a) - np.log(b))).sum()

def rollout_step(env, log_p_c, theta, pi, 
                 use_info_gain, use_pragmatic_value):
    
    if pi == []:
        return []

    a, pi_rest = pi[0], pi[1:]

    # Where will I be after taking action a?
    theta1 = update_belief_a(env, theta, a=a, lr=1.) 
    q = softmax(theta1)

    # Do I like being there?
    pragmatic = np.dot(q, log_p_c)

    # What might I observe after taking action a? (marginalize p(o, s) over s)
    p_o = np.dot(q, env.p_o_given_s)

    # Do I learn about s from by observing o?
    # enumerate/ sample observations, update belief and estimate info gain
    q_o = [softmax(update_belief(env, theta1, o=i)) for i in range(p_o.shape[0])]
    d_o = [kl(q_o_i, q) for q_o_i in q_o] # info gain for each observation
    info_gain = np.dot(p_o, d_o) # expected value of info gain

    # negative expected free energy for this timestep
    nefe = use_pragmatic_value * pragmatic + use_info_gain * info_gain

    # nefe for remainder of policy rollout
    nefe_rest = rollout_step(env, log_p_c, theta1, pi_rest, 
                        use_info_gain=use_info_gain, 
                        use_pragmatic_value=use_pragmatic_value)

    # concatenate expected free energy across future time steps
    return [nefe] + nefe_rest

def select_action(env, theta_star, theta_start, 
                  k=4, # planning horizon (number of sequential actions per plan)
                  n_actions=2, # possible actions (assumed to be discrete and indexed 0)
                  use_info_gain=True, 
                  use_pragmatic_value=True,
                  select_max_pi=False, # replace sampling with best action selection
                  debug=False, # return plans, p of selecting each, and marginal p of actions
                 ):
    log_p_c = np.log(softmax(theta_star))

    # sampling
    #n_plans = 32
    #plans = np.random.choice(n_actions, size=(n_plans, k), replace=True).tolist()
    # genrate all plans
    plans = [ list(x) for x in itertools.product(range(n_actions), repeat=k)]

    # evaluate negative expected free energy of all plans
    nefes = []
    for pi in plans:
        step_nefes = rollout_step(env, log_p_c, theta_start, pi, 
                                  use_info_gain=use_info_gain, 
                                  use_pragmatic_value=use_pragmatic_value)
        nefe = np.array(step_nefes).mean() # expected value over steps
        nefes.append(nefe)
        
    # compute probability of following each plan
    p_pi = softmax(np.array(nefes)).tolist()  

    if select_max_pi:
        a = plans[np.argmax(nefes)]
    else:
        a = plans[np.random.choice(len(plans), p=p_pi)]
    
    if debug:
        p_a = np.zeros(n_actions)
        for p, pi in zip(p_pi, plans):
            p_a[pi[0]] += p
            
        return a, p_a, plans, p_pi
    
    return a

Let's explore action selection from plans with horizon $k$ by specifying sharp priors on the starting state and target state $k-1$ steps apart.

If the starting state is to the right of the target (recall the state space wraps around), then policies that take a sequence of left actions ($a=0$)) are scored higher. Note that this holds true irrespective of the food source location. 

If the starting state is to the left of the target (e.g., $s_0=11$), then policies that take a sequence of right actions ($a=1$) are scored higher.

If the starting state and the target state coincide, then policies that take equal numbers of left and right actions are scored highest.



In [ ]:
starting_state = 11
target_state = 14


env = me.MinimalEnv(N=16, # number of states
                    s_food=8) # location of the food source

# initialize belief
theta_start = np.eye(env.s_N)[starting_state] * 10 # believe we are in state 11

# initialize preference
theta_star = np.eye(env.s_N)[target_state] * 10

a, p_a, plans, p_pi = select_action(env, theta_star, theta_start, debug=True)

# and explore what the agent prefers
plt.bar(x = range(len(plans)), height=p_pi)
plt.xlabel('plan id')
plt.ylabel('$p(\pi)$')

print('plans and associated probability of selecting them.')
for p, pi in zip(p_pi, plans):
    print(pi, p)

# estimate marginal probability of selecting a plan with first action 0 or 1
print('marginal probability of next action')
print(p_a)

## Putting it all together

Now we have all components required to define a complete Active Infererence agent. Let's encapsulate it into a class that manages the target state and current belief state over time and provides a minimal interface with reset and step methods.


In [ ]:
class MinimalAgent:
    
    def __init__(self, 
                 env,
                 target_state, 
                 k=2, # planning horizon
                 use_info_gain=True, # score actions by info gain
                 use_pragmatic_value=True, # score actions by pragmatic value
                 select_max_pi=False, # sample plan (False), select max negEFE (True).
                 n_steps_o=20, # optimization steps after new observation
                 n_steps_a=20, # optimization steps after new action
                 lr_o=4., # learning rate of optimization after new observation
                 lr_a=4.): # learning rate of optimization after new action)
        
        self.env = env
        self.target_state = target_state
        self.k = k
        self.use_info_gain = use_info_gain
        self.use_pragmatic_value = use_pragmatic_value
        self.select_max_pi = select_max_pi
        self.n_steps_o = n_steps_o
        self.n_steps_a = n_steps_a
        self.lr_a = lr_a
        self.lr_o = lr_o
        
    def reset(self):
        # initialize state preference
        self.b_star = np.eye(self.env.s_N)[self.target_state] * 10
        self.log_p_c = np.log(softmax(self.b_star))
        # initialize state prior as uniform
        self.b = np.zeros(self.env.s_N)
        
    def step(self, o, debug=False):
        if debug:
            return self._step_debug(o)
        
        self.b = self._update_belief(theta_prev=self.b, o=int(o))
        a = select_action(theta_start=self.b)[0] # pop first action of selected plan
        self.b = self._update_belief_a(theta_prev=self.b, a=a)
        return a
    
    def _step_debug(self, o):
        self.b, ll_o = self._update_belief(theta_prev=self.b, 
                                           o=int(o), debug=True)
        a, p_a, _, _ = self._select_action(theta_start=self.b, debug=True)
        a = a[0]
        self.b, ll_a = self._update_belief_a(theta_prev=self.b, a=a, debug=True)
        return a, ll_o, ll_a, p_a
    
    def _update_belief_a(self, theta_prev, a, debug=False):
        # prior assumed to be expressed as parameters of the softmax (logits)
        theta = torch.tensor(theta_prev)
        q = torch.nn.Softmax(dim=0)(theta)

        # this is the prior for the distribution at time t
        q1 = torch.matmul(q, torch.tensor(self.env.p_s1_given_s_a[:,a,:]))

        # initialize parameters of updated belief to uniform
        theta1 = torch.zeros_like(theta, requires_grad=True)
        loss = torch.nn.CrossEntropyLoss() # expects logits and target distribution.
        optimizer = torch.optim.SGD([theta1], lr=self.lr_a)
        if debug:
            ll = np.zeros(self.n_steps_a)

        for i in range(self.n_steps_a):
            l = loss(theta1, q1)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            if debug:
                ll[i] = l.detach().numpy()

        theta1 = theta1.detach().numpy()
        if debug:
            return theta1, ll

        return theta1
    
    def _update_belief(self, theta_prev, o, debug=False):
        theta = torch.tensor(theta_prev)

        # make p(s) from b
        q = torch.nn.Softmax(dim=0)
        p = torch.tensor(self.env.p_o_given_s[:,o]) * q(theta) # p(o|s)p(s)
        log_p = torch.log(p)

        # initialize updated belief with current belief
        theta1 = torch.tensor(theta_prev, requires_grad=True)

        # estimate loss
        def forward():
            q1 = q(theta1)
            # free energy: KL[ q(s) || p(s, o) ]
            fe = torch.sum(q1 * (torch.log(q1) - log_p))
            return fe

        optimizer = torch.optim.SGD([theta1], lr=self.lr_o)
        ll = np.zeros(self.n_steps_o)
        for i in range(self.n_steps_o):
            l = forward()
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            if debug:
                ll[i] = l.detach().numpy()

        theta1 = theta1.detach().numpy()
        if debug:
            return theta1, ll

        return theta1

    def _select_action(self, theta_start,
                       debug=False): # return plans, p of selecting each, and marginal p of actions
        # sampling
        #n_plans = 32
        #plans = np.random.choice(n_actions, size=(n_plans, k), replace=True).tolist()
        # genrate all plans
        plans = [ list(x) for x in itertools.product(range(self.env.a_N), repeat=self.k)]
        # evaluate negative expected free energy of all plans
        nefes = []
        for pi in plans:
            step_nefes = self._rollout_step(theta_start, pi)
            nefe = np.array(step_nefes).mean() # expected value over steps
            nefes.append(nefe)

        # compute probability of following each plan
        p_pi = softmax(np.array(nefes)).tolist()
        if self.select_max_pi:
            a = plans[np.argmax(nefes)]
        else:
            a = plans[np.random.choice(len(plans), p=p_pi)]

        if debug:
            # compute marginal action probabilities
            p_a = np.zeros(self.env.a_N)
            for p, pi in zip(p_pi, plans):
                p_a[pi[0]] += p

            return a, p_a, plans, p_pi

        return a

    def _rollout_step(self, theta, pi):
        if pi == []:
            return []

        a, pi_rest = pi[0], pi[1:]
        # Where will I be after taking action a?
        theta1 = self._update_belief_a(theta, a=a) 
        q = softmax(theta1)
        # Do I like being there?
        pragmatic = np.dot(q, self.log_p_c)
        # What might I observe after taking action a? (marginalize p(o, s) over s)
        p_o = np.dot(q, self.env.p_o_given_s)
        # Do I learn about s from by observing o?
        # enumerate/ sample observations, update belief and estimate info gain
        q_o = [softmax(self._update_belief(theta1, o=i)) for i in range(p_o.shape[0])]
        d_o = [kl(q_o_i, q) for q_o_i in q_o] # info gain for each observation
        info_gain = np.dot(p_o, d_o) # expected value of info gain
        # negative expected free energy for this timestep
        nefe = self.use_pragmatic_value * pragmatic + \
               self.use_info_gain * info_gain
        # nefe for remainder of policy rollout
        nefe_rest = self._rollout_step(theta1, pi_rest)
        # concatenate expected free energy across future time steps
        return [nefe] + nefe_rest


The code below iterates over all steps involved in the interaction between the environment and the active inference agent. In each interaction step, the agent updates its belief about the current state given a new observation and selects an action to minimise expected free energy. It then updates its belief assuming the selected action was taken and starts anew by updating its belief based on the next observation.

In [ ]:
import importlib
import minimal_agent_torch_vi as ma
importlib.reload(ma)

target_state = 4
k = 4 # planning horizon; run time increases exponentially with planning horizon

# runtime increases linearly with optimization steps during belief update
n_steps_o = 20 # optimization steps updating belief after observation
n_steps_a = 20 # optimization steps updating belief after action
lr_o = 4. # learning rate updating belief after observation
lr_a = 4. # learning rate updating belief after action

render_losses = True

env = me.MinimalEnv(N=16, # number of states
                    s_food=0, # location of the food source
                    s_0=0) # starting location 

agent = ma.MinimalAgentVI(env=env, 
                     target_state=target_state, 
                     k=k, 
                     use_info_gain=True,
                     use_pragmatic_value=True,
                     select_max_pi=False,
                     n_steps_o=n_steps_o, 
                     n_steps_a=n_steps_a, 
                     lr_a=lr_a, 
                     lr_o=lr_o)

o = env.reset() # set state to starting state
agent.reset() # initialize belief state and target state distribution

ss = [env.s_t]
bb = [agent.b]
aa = []
if render_losses:
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].set_title('updates from actions')
    ax[0].set_ylabel('loss')
    ax[0].set_xlabel('optimization step')
    ax[1].set_title('updates from observations')
    ax[1].set_ylabel('loss')
    ax[1].set_xlabel('optimization step')
    
for i in range(64):
    a, ll_o, ll_a, p_a = agent.step(o, debug=True)
    print(f"step {i}, s: {env.s_t}, o: {['FOOD', 'NONE'][int(o)]}, p(a): {p_a}, a: {['UP', 'DOWN'][a]}")
    if render_losses:
        ax[0].plot(ll_a)
        ax[1].plot(ll_o)
    
    o = env.step(a)
    
    ss.append(env.s_t)
    bb.append(agent.b)
    aa.append(a)


from matplotlib.markers import CARETUP, CARETDOWN
aa = np.array(aa)
ss = np.array(ss)

fig, ax = plt.subplots(figsize=(16, 6))
plt.imshow(np.array(bb).T, label='belief')
t = np.arange(len(aa))
i_left = t[aa==0]
i_right = t[aa==1]
plt.scatter(i_left, ss[:-1][i_left], c='white', marker=CARETUP)
plt.scatter(i_right, ss[:-1][i_right], c='white', marker=CARETDOWN)
plt.plot(ss, label='state')
plt.plot([0, len(ss)-1], [target_state]*2, label='target')
plt.plot([0, len(ss)-1], [env.s_food]*2, 'w--', label='food')
plt.legend()

## Summary

In this notebook we developed the individual components of an active inference agent that are responsible for updating the agents belief after a new observation arrived and after a new action was taken, and for selecting actions based on the expected free energy, which we separated into an infomation gain term and a pragmatic value term. Each of these components were tested against expected behavior and diagnostic visualisations were introduced that can help adjust learning rates and number of optimization steps and that let us interpret action selection based on the marginal probability of a first action across all evaluated plans.

We put everything together, encapsulated properties, state and methods in the `MinimalAgent` class, and analyzed this agent's sequential interactions with the `MinimalEnvironment`.

There may be potential for further optimizing the code by separating torch compute graph definitions and their application to optimizing each individual interaction step's data.

## Outlook

The agentwe  developed above can interact with environments with
- discrete state spaces
- discrete observation spaces
- discrete action spaces

In subsequent notebooks, we will modify the agent to enable interaction with environments that have

1. continuous action spaces
2. continuous observation spaces
3. continuous state spaces